In [1]:
# The code was removed by Watson Studio for sharing.

In [18]:
! pip install featuretools

     |████████████████████████████████| 276kB 7.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 16.9MB/s eta 0:00:01
  Found existing installation: tqdm 4.31.1
    Uninstalling tqdm-4.31.1:
      Successfully uninstalled tqdm-4.31.1


In [2]:
# Load Package
# import numpy as np
import pandas as pd
import seaborn as sns

In [19]:
import featuretools as ft

In [53]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    LabelEncoder,
    OrdinalEncoder,
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

In [4]:
# Load data
train_df = project.get_file("train.csv")
test_df = project.get_file("test.csv")
train_df= pd.read_csv(train_df)
test_df = pd.read_csv(test_df)

In [5]:
# Save data
target = train_df[["Survived"]]
submission = test_df[["PassengerId"]]

In [104]:
# Join and Clean
combine = pd.concat([train_df, test_df])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [105]:
# EDA
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [106]:
combine.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')

In [107]:
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

In [108]:
combine['Title'] = combine.Name.apply(lambda x: x.split(".")[0].split(",")[1].strip()).replace(mapping)

In [109]:
combine.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

In [110]:
# combine['Sex2'] = combine['Sex'].apply(lambda x: 0 if x=='female' else 1)

In [111]:
# class ModifiedLabelEncoder(LabelEncoder):

#     def fit_transform(self, y, *args, **kwargs):
#         return super().fit_transform(y)

#     def transform(self, y, *args, **kwargs):
#         return super().transform(y)

In [112]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encode", OrdinalEncoder()),
])

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

In [157]:
combine[['Sex','Embarked', 'Title']] = categorical_transformer.fit_transform(combine[['Sex', 'Embarked', 'Title']])

In [158]:
combine[['Age', 'Fare']] = numeric_transformer.fit_transform(combine[['Age', 'Fare']])

In [159]:
es = ft.EntitySet(id = 'titanic_data')

es = es.entity_from_dataframe(entity_id = 'combine', dataframe = combine.drop(['Survived'], axis=1), 
                              variable_types = 
                              {
                                  'Embarked': ft.variable_types.Categorical,
                                  'Sex': ft.variable_types.Boolean,
                                  'Title': ft.variable_types.Categorical
                              },
                              index = 'PassengerId')

es

Entityset: titanic_data
  Entities:
    combine [Rows: 1309, Columns: 9]
  Relationships:
    No relationships

In [160]:
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Embarked', index='Embarked')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Sex', index='Sex')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Title', index='Title')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Pclass', index='Pclass')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Parch', index='Parch')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='SibSp', index='SibSp')
es

Entityset: titanic_data
  Entities:
    combine [Rows: 1309, Columns: 9]
    Embarked [Rows: 3, Columns: 1]
    Sex [Rows: 2, Columns: 1]
    Title [Rows: 7, Columns: 1]
    Pclass [Rows: 3, Columns: 1]
    Parch [Rows: 8, Columns: 1]
    SibSp [Rows: 7, Columns: 1]
  Relationships:
    combine.Embarked -> Embarked.Embarked
    combine.Sex -> Sex.Sex
    combine.Title -> Title.Title
    combine.Pclass -> Pclass.Pclass
    combine.Parch -> Parch.Parch
    combine.SibSp -> SibSp.SibSp

In [161]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(primitives[primitives['type'] == 'aggregation'].shape[0])

,name,type,dask_compatible,description
0,trend,aggregation,False,Calculates the trend of a variable over time.
1,count,aggregation,True,"Determines the total number of values, excluding `NaN`."
2,n_most_common,aggregation,False,Determines the `n` most common elements.
3,num_unique,aggregation,True,"Determines the number of distinct values, ignoring `NaN` values."
4,num_true,aggregation,True,Counts the number of `True` values.
5,min,aggregation,True,"Calculates the smallest value, ignoring `NaN` values."
6,all,aggregation,True,Calculates if all values are 'True' in a list.
7,entropy,aggregation,False,Calculates the entropy for a categorical variable
8,mean,aggregation,True,Computes the average for a list of values.
9,last,aggregation,False,Determines the last value in a list.


In [162]:
primitives[primitives['type'] == 'transform'].head(primitives[primitives['type'] == 'transform'].shape[0])

,name,type,dask_compatible,description
22,multiply_boolean,transform,True,Element-wise multiplication of two lists of boolean values.
23,time_since_previous,transform,False,Compute the time since the previous entry in a list.
24,not_equal_scalar,transform,True,Determines if values in a list are not equal to a given scalar.
25,greater_than_scalar,transform,True,Determines if values are greater than a given scalar.
26,and,transform,True,Element-wise logical AND of two lists.
27,is_null,transform,True,Determines if a value is null.
28,multiply_numeric,transform,True,Element-wise multiplication of two lists.
29,modulo_numeric_scalar,transform,True,Return the modulo of each element in the list by a scalar.
30,subtract_numeric_scalar,transform,True,Subtract a scalar from each element in the list.
31,divide_numeric_scalar,transform,True,Divide each element in the list by a scalar.


In [163]:
features, feature_names = ft.dfs(entityset = es, 
                                 target_entity = 'combine', 
#                                  trans_primitives=['subtract_numeric', 'add_numeric', 'divide_numeric', 'multiply_numeric'],
                                 max_depth = 2)

In [164]:
feature_names

[<Feature: Age>,
 <Feature: Fare>,
 <Feature: Parch>,
 <Feature: Pclass>,
 <Feature: SibSp>,
 <Feature: Embarked>,
 <Feature: Sex>,
 <Feature: Title>,
 <Feature: Embarked.SUM(combine.Age)>,
 <Feature: Embarked.SUM(combine.Fare)>,
 <Feature: Embarked.STD(combine.Age)>,
 <Feature: Embarked.STD(combine.Fare)>,
 <Feature: Embarked.MAX(combine.Age)>,
 <Feature: Embarked.MAX(combine.Fare)>,
 <Feature: Embarked.SKEW(combine.Age)>,
 <Feature: Embarked.SKEW(combine.Fare)>,
 <Feature: Embarked.MIN(combine.Age)>,
 <Feature: Embarked.MIN(combine.Fare)>,
 <Feature: Embarked.MEAN(combine.Age)>,
 <Feature: Embarked.MEAN(combine.Fare)>,
 <Feature: Embarked.COUNT(combine)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Pclass)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Sex)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Title)>,
 <Feature: Embarked.NUM_UNIQUE(combine.SibSp)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Parch)>,
 <Feature: Embarked.MODE(combine.Pclass)>,
 <Feature: Embarked.MODE(combine.Sex)>,
 <Feature: E

In [165]:
len(feature_names)

146

In [166]:
features.isnull().sum()

Age                                    0
Fare                                   0
Parch                                  0
Pclass                                 0
SibSp                                  0
Embarked                               0
Sex                                    0
Title                                  0
Embarked.SUM(combine.Age)              0
Embarked.SUM(combine.Fare)             0
Embarked.STD(combine.Age)              0
Embarked.STD(combine.Fare)             0
Embarked.MAX(combine.Age)              0
Embarked.MAX(combine.Fare)             0
Embarked.SKEW(combine.Age)             0
Embarked.SKEW(combine.Fare)            0
Embarked.MIN(combine.Age)              0
Embarked.MIN(combine.Fare)             0
Embarked.MEAN(combine.Age)             0
Embarked.MEAN(combine.Fare)            0
Embarked.COUNT(combine)                0
Embarked.NUM_UNIQUE(combine.Pclass)    0
Embarked.NUM_UNIQUE(combine.Sex)       0
Embarked.NUM_UNIQUE(combine.Title)     0
Embarked.NUM_UNI

In [198]:
class RemoveLowInfo(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = pd.DataFrame(X)
        keep = [column for column in df.columns if df[column].value_counts(normalize=True).reset_index(drop=True)[0]<self.threshold]
        return df[keep]

In [199]:
from sklearn.decomposition import PCA

In [200]:
transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ('removelowinfo',RemoveLowInfo(threshold=.95)),
    ('pca', PCA()),
])

In [201]:
features = transformer.fit_transform(features)

In [193]:
features

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,138,140,142
0,22.0,7.2500,0.0,3.0,1.0,2.0,1.0,3.0,26665.75,25207.8404,...,0.152625,2.239283,0.17,6.4375,30.684201,48.711300,319.0,6.0,8.0,0.0
1,38.0,71.2833,0.0,1.0,1.0,0.0,0.0,4.0,8478.42,16830.7922,...,0.152625,2.239283,0.17,6.4375,30.684201,48.711300,319.0,6.0,8.0,0.0
2,26.0,7.9250,0.0,3.0,0.0,2.0,0.0,2.0,26665.75,25207.8404,...,0.982743,5.871926,0.33,0.0000,30.246251,25.772689,891.0,7.0,6.0,1.0
3,35.0,53.1000,0.0,1.0,1.0,2.0,0.0,4.0,26665.75,25207.8404,...,0.152625,2.239283,0.17,6.4375,30.684201,48.711300,319.0,6.0,8.0,0.0
4,35.0,8.0500,0.0,3.0,0.0,2.0,1.0,3.0,26665.75,25207.8404,...,0.982743,5.871926,0.33,0.0000,30.246251,25.772689,891.0,7.0,6.0,1.0
5,28.0,8.4583,0.0,3.0,0.0,1.0,1.0,3.0,3475.50,1526.3085,...,0.982743,5.871926,0.33,0.0000,30.246251,25.772689,891.0,7.0,6.0,1.0
6,54.0,51.8625,0.0,1.0,0.0,2.0,1.0,3.0,26665.75,25207.8404,...,0.982743,5.871926,0.33,0.0000,30.246251,25.772689,891.0,7.0,6.0,1.0
7,2.0,21.0750,1.0,3.0,3.0,2.0,1.0,1.0,26665.75,25207.8404,...,-0.351280,1.619736,2.00,15.8500,18.650000,71.332090,20.0,4.0,3.0,0.0
8,27.0,11.1333,2.0,3.0,0.0,2.0,0.0,4.0,26665.75,25207.8404,...,0.982743,5.871926,0.33,0.0000,30.246251,25.772689,891.0,7.0,6.0,1.0
9,14.0,30.0708,0.0,2.0,1.0,0.0,0.0,4.0,8478.42,16830.7922,...,0.152625,2.239283,0.17,6.4375,30.684201,48.711300,319.0,6.0,8.0,0.0
